In [1]:
import torch
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

In [2]:
# load images from given folder
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images


datas = load_images('data')
print("Total images: ", len(datas))
print("Image shape: ", datas[0].shape)

Total images:  200
Image shape:  (256, 256, 3)


In [20]:
# extract features from given image data using 16x16 grid histogram for each channel
grid_size = 64
def extract_features(data):
    features = []
    for img in data:
        hist = []
        for i in range(3):
            for x in range(0, 256, grid_size):
                for y in range(0, 256, grid_size):
                    hist.append(cv2.calcHist([img[x:x+grid_size, y:y+grid_size, i]], [0], None, [256], [0, 256]).argmax())

        features.append(hist)
    return pd.DataFrame(features)


df = extract_features(datas)
output=[0]*100 + [1]*100
column_names = []
for k in range(1, 17):
    column_names.extend([f'R{k}', f'G{k}', f'B{k}'])
df.columns = column_names
df['target']=output
df.to_csv('features.csv', index=False)
df.head()

,R1,G1,B1,R2,G2,B2,R3,G3,B3,R4,...,R14,G14,B14,R15,G15,B15,R16,G16,B16,target
0,103,111,109,108,121,111,113,113,116,109,...,110,114,112,116,115,127,116,112,125,0
1,115,122,116,111,114,115,117,117,114,111,...,116,116,114,113,112,121,116,126,120,0
2,115,103,111,109,114,110,107,105,109,109,...,116,120,120,120,119,113,111,117,117,0
3,116,117,116,121,108,106,106,102,107,105,...,105,107,103,103,104,107,103,101,103,0
4,144,142,149,149,151,149,144,153,147,150,...,143,157,148,147,138,162,174,152,143,0


In [21]:
# Classification using MLP model with sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('features.csv')
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))


Accuracy:  0.9


In [22]:
class ImageDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.tensor(data.values).float()
        self.target = torch.tensor(target.values).long()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

X = df.drop('target', axis=1)
y = df['target']

# Classification using MLP model with PyTorch
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
train_dataset = ImageDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = ImageDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(48, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epoch = 10000

for epoch in range(epoch):
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


with torch.no_grad():
    correct = 0
    total = 0
    for data, target in test_dataloader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
    

print('Accuracy: ', correct / total)

Epoch 0, Loss: 3.229811668395996
Epoch 100, Loss: 0.1266646832227707
Epoch 200, Loss: 0.03946523740887642
Epoch 300, Loss: 0.002832624129951
Epoch 400, Loss: 0.02627333626151085
Epoch 500, Loss: 0.0020359810441732407
Epoch 600, Loss: 0.001381992595270276
Epoch 700, Loss: 0.0004701522411778569
Epoch 800, Loss: 0.00029754918068647385
Epoch 900, Loss: 3.972342892666347e-05
Epoch 1000, Loss: 4.2079474951606244e-05
Epoch 1100, Loss: 2.5982757506426424e-05
Epoch 1200, Loss: 8.571397302148398e-06
Epoch 1300, Loss: 3.6283847748563858e-06
Epoch 1400, Loss: 2.931761173385894e-06
Epoch 1500, Loss: 4.030667696497403e-06
Epoch 1600, Loss: 3.3452643037890084e-06
Epoch 1700, Loss: 1.2181629926999449e-06
Epoch 1800, Loss: 1.1958115919696866e-06
Epoch 1900, Loss: 1.8514560906623956e-06
Epoch 2000, Loss: 2.4474838937749155e-06
Epoch 2100, Loss: 6.817248845436552e-07
Epoch 2200, Loss: 1.676379355330937e-07
Epoch 2300, Loss: 1.6018742599044344e-07
Epoch 2400, Loss: 7.376058874797309e-07
Epoch 2500, Loss: 